In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math 
from collections import defaultdict

In [50]:
complete_data = pd.read_csv("datathon_2024_dataset.csv")

In [51]:
complete_data = complete_data[['game_date','home_team','away_team','city']]

In [38]:
import requests
from pytz import timezone

def get_timezone(latitude, longitude):
    api_key = "83ETYBRSZ5G1"
    url = f"http://api.timezonedb.com/v2.1/get-time-zone?key=83ETYBRSZ5G1&format=json&by=position&lat={latitude}&lng={longitude}"
    response = requests.get(url,timeout = 100)
    if response.status_code == 200:
        timezone_data = response.json()
        gmt_offset_seconds = timezone_data["gmtOffset"]
        gmt_offset_hours = gmt_offset_seconds / 3600  # Convert seconds to hours
        #timezone_id = f"Etc/GMT{int(gmt_offset_hours)}"
        #return timezone(timezone_id)
        return gmt_offset_hours
    else:
        print(f"Error: {response.status_code}")
        return None

# Example usage
#timezone_object = get_timezone("40.6", "-64.0")  # Latitude and longitude for London
#if timezone_object:
    #print(timezone_object.zone)  # Output: Europe/London
#print(timezone_object)

In [39]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
geolocator = Nominatim(user_agent="MyGeocoder/1.0.0 (kig3@rice.edu)",timeout=100)

location_dataset = []
unique_values = data['city'].unique()

for city in unique_values:
    loc = geolocator.geocode(city)
    
    # Check if location is found before accessing latitude and longitude
    if loc:
        location_data = {
            'city': city,
            'latitude': loc.latitude,
            'longitude': loc.longitude
        }
        location_dataset.append(location_data)
location_dataset = pd.DataFrame(location_dataset)

In [40]:
import time

In [46]:
for idx,row in location_dataset.iterrows():
    k = get_timezone(row['latitude'],row['longitude'])
    print(k)
    location_dataset.at[idx,'gmtOffset'] = k
    time.sleep(2)
print(location_dataset)

9.0
-5.0
-5.0
-5.0
-5.0
-5.0
-6.0
-8.0
-5.0
-6.0
-8.0
-6.0
-5.0
-7.0
-5.0
-8.0
-6.0
-6.0
3.0
-6.0
-7.0
-6.0
-5.0
-8.0
-8.0
-5.0
-5.0
-8.0
-5.0
-4.0
-5.0
-5.0
11.0
-8.0
-5.0
-6.0
-6.0
0.0
-6.0
13.0
-6.0
                city   latitude   longitude  gmtOffset
0              Tokyo  35.682194  139.762221        9.0
1            Atlanta  33.748992  -84.390264       -5.0
2         Cincinnati  39.101454  -84.512460       -5.0
3              Miami  25.774173  -80.193620       -5.0
4           Montreal  45.503182  -73.569806       -5.0
5           New York  40.712728  -74.006015       -5.0
6          St. Louis  38.628028  -90.191015       -6.0
7            Anaheim  33.834752 -117.911732       -8.0
8          Baltimore  39.290882  -76.610759       -5.0
9        Minneapolis  44.977300  -93.265469       -6.0
10           Oakland  37.804456 -122.271356       -8.0
11         Arlington  32.735582  -97.107119       -6.0
12           Toronto  43.653482  -79.383935       -5.0
13           Phoenix  33.448

In [47]:
location_dataset.to_csv("location_dataset.csv")

In [53]:
final_data = []
complete_data['game_date'] = complete_data['game_date'].astype(str)
for i in range(2000,2024):
    year = str(i)
    data = complete_data[complete_data['game_date'].str[:4] == year]
    data['game_date'] = pd.to_datetime(data['game_date'], format='%Y%m%d')
    timezone_map = defaultdict(int)
    for index, row in data.iterrows():
        curr_location = location_dataset.loc[location_dataset['city'] == row['city']].iloc[0]
        curr_timezone = curr_location['gmtOffset']
        if timezone_map[row['home_team']] == 0:
            data.at[index, 'time_diff_from_last_location_home_team'] = 0
        else:
            data.at[index, 'time_diff_from_last_location_home_team'] = curr_timezone - timezone_map[row['home_team']] 
        timezone_map[row['home_team']] = curr_timezone

        if timezone_map[row['away_team']] == 0:
            data.at[index, 'time_diff_from_last_location_away_team'] = 0
        else:
            data.at[index, 'time_diff_from_last_location_away_team'] = curr_timezone - timezone_map[row['away_team']]
        timezone_map[row['away_team']] = curr_timezone
    final_data.append(data)
final_df = pd.concat(final_data, ignore_index=True) 

C:\Users\kusha\AppData\Local\Temp\ipykernel_22048\3979459707.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['game_date'] = pd.to_datetime(data['game_date'], format='%Y%m%d')
C:\Users\kusha\anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\kusha\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [56]:
print(final_df.head(75))

    game_date home_team away_team        city  \
0  2000-03-29       NYN       CHN       Tokyo   
1  2000-03-30       CHN       NYN       Tokyo   
2  2000-04-03       ATL       COL     Atlanta   
3  2000-04-03       CIN       MIL  Cincinnati   
4  2000-04-03       FLO       SFN       Miami   
..        ...       ...       ...         ...   
70 2000-04-08       CIN       CHN  Cincinnati   
71 2000-04-08       FLO       COL       Miami   
72 2000-04-08       HOU       PHI     Houston   
73 2000-04-08       MON       SDN    Montreal   
74 2000-04-08       NYN       LAN    New York   

    time_diff_from_last_location_home_team  \
0                                      0.0   
1                                      0.0   
2                                      0.0   
3                                      0.0   
4                                      0.0   
..                                     ...   
70                                     0.0   
71                                     0.0 

In [58]:
final_df.to_csv('timezone_data.csv')